In [51]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from collections import Counter
import re
from part_of_speech import get_part_of_speech






In [47]:
df = pd.read_csv('../okcupiddata/profiles.csv')
df.columns

Index(['age', 'body_type', 'diet', 'drinks', 'drugs', 'education', 'essay0',
       'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7',
       'essay8', 'essay9', 'ethnicity', 'height', 'income', 'job',
       'last_online', 'location', 'offspring', 'orientation', 'pets',
       'religion', 'sex', 'sign', 'smokes', 'speaks', 'status'],
      dtype='object')

In [48]:
#make dataframe just containing essays
df_essays = df[['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9']]
#df_essays['corpus'] = df_essays[df_essays.columns[0:]].apply(
#    lambda x: ','.join(x.dropna().astype(str)),
#    axis=1)
df_essays.head()

,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,about me:<br />\n<br />\ni would love to think...,currently working as an international agent fo...,making people laugh.<br />\nranting about a go...,"the way i look. i am a six foot half asian, ha...","books:<br />\nabsurdistan, the republic, of mi...",food.<br />\nwater.<br />\ncell phone.<br />\n...,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet!<br />\nyou...
1,i am a chef: this is what that means.<br />\n1...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories.<br /...,NaN,NaN,i am very open and will share just about anyth...,NaN
2,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement<br />\nconversation<br />\ncreation<b...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ..."
3,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . .<br />\nlynch, j...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.
4,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at:<br />\nhttp://bag...,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians<br />\nat the...",NaN,NaN,NaN,NaN,NaN


In [66]:
def remove_noise(text):
    #remove html tags
    text = text.str.replace(r'<[^<]+?>', '')
    #remove punctuation
    text = text.str.replace(r'[\.\?\!\,\:\;\"]', '')
    #remove newline, tab
    text = text.str.replace(r'\r+|\n+|\t+',' ')
    return text

def tokenize_lemmatize_remove_stop(text):
    tokenized = word_tokenize(str(text))
    #print(tokenized)
    lemmatized = [lemmatizer.lemmatize(token, get_part_of_speech(token)) for token in tokenized]
    no_stop = [word for word in lemmatized if word not in stop_words]
    return no_stop

denoised = df_essays.apply(lambda row: remove_noise(row))
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
denoised = pd.DataFrame(denoised)
normalized = denoised.apply(lambda row: tokenize_lemmatize_remove_stop(row))
normalized = pd.DataFrame(normalized)

print(len(normalized[0][0]))

79


In [69]:
print(len(denoised))

59946
